# Mobility-Station-Finder

## Imports

In [1]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from matrixconverters.read_ptv import ReadPTVMatrix
import requests

## Read and process static data

### Paths

In [2]:
path_npvm_zones = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\Verkehrszonen_Schweiz_NPVM_2017_shp.zip"
path_mobility_stations = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\mobility-stationen-und-fahrzeuge-schweiz.csv"
path_pt_jrta = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\140_JRTA_(OEV).mtx"
path_pt_ntr = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\144_NTR_(OEV).mtx"

### Read NPVM-zones with shapes

In [3]:
%time gdf_npvm_zones = gpd.read_file(path_npvm_zones, encoding="cp1252").to_crs(4326)

CPU times: total: 3.52 s
Wall time: 3.51 s


In [4]:
gdf_npvm_zones.head()

,ID,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR,geometry
0,101001,1,1,Aeugst am Albis,1,None,1,ZH,3,Ländlich,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.47334 47.26128, 8.47334 47.26139, ..."
1,201001,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.42224 47.29775, 8.42282 47.29816, ..."
2,201002,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.44770 47.26794, 8.44767 47.26782, ..."
3,201003,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.43834 47.27714, 8.43814 47.27726, ..."
4,201004,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.45000 47.27949, 8.45007 47.27945, ..."


In [5]:
print("Anzahl NPVM-Zonen: {}".format(len(gdf_npvm_zones)))

Anzahl NPVM-Zonen: 7978


In [6]:
def get_npvm_zone(id_):
    return gdf_npvm_zones[gdf_npvm_zones.ID == id_]

In [7]:
get_npvm_zone(5301003)

,ID,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR,geometry
84,5301003,53,53,Bülach,1,None,1,ZH,1,Städtisch,261,Zürich,12032,Kloten,"POLYGON ((8.53973 47.51618, 8.54007 47.51529, ..."


### Read Mobility-stations, assign NPVM-zones to Mobility-stations

In [8]:
df_mobility_vechicles = pd.read_csv(path_mobility_stations, delimiter=";", encoding="utf8")[["Stationsnummer", "Name", "Standort"]].dropna()

In [9]:
print("Anzahl Mobility Fahrzeuge: {}".format(len(df_mobility_vechicles)))

Anzahl Mobility Fahrzeuge: 2662


In [10]:
df_mobility_stations = df_mobility_vechicles.groupby("Stationsnummer").first().reset_index()

In [11]:
df_mobility_stations["lon"] = df_mobility_stations["Standort"].apply(lambda x: x.split(",")[1])
df_mobility_stations["lat"] = df_mobility_stations["Standort"].apply(lambda x: x.split(",")[0])
df_mobility_stations = gpd.GeoDataFrame(df_mobility_stations, geometry=gpd.points_from_xy(df_mobility_stations.lon, df_mobility_stations.lat), crs=4326)

In [12]:
print("Anzahl Mobility Stationen: {}".format(len(df_mobility_stations)))

Anzahl Mobility Stationen: 1550


In [13]:
gdf_mobilty_stations_with_npvm_zone = gpd.sjoin(df_mobility_stations, gdf_npvm_zones)[["Stationsnummer", "Name", "geometry", "ID", "N_Gem"]]

In [14]:
print("Anzahl Mobility Standorte mit zugeordneter NPVM-Zone: {}".format(len(gdf_mobilty_stations_with_npvm_zone)))

Anzahl Mobility Standorte mit zugeordneter NPVM-Zone: 1549


In [15]:
gdf_mobilty_stations_with_npvm_zone.head()

,Stationsnummer,Name,geometry,ID,N_Gem
0,1006,Brugg Bahnhof,POINT (8.20942 47.48154),409501008,Brugg
719,3215,Brugg Post Neumarkt / Bahnhofstrasse,POINT (8.20757 47.48216),409501008,Brugg
1,1012,Arbon Bahnhof,POINT (9.43345 47.51032),440101010,Arbon
2,1019,Basel Vogesenstrasse,POINT (7.57483 47.56869),270101028,Basel
3,1024,Bellinzona Stazione,POINT (9.03017 46.19630),500201015,Bellinzona


In [16]:
gdf_npvm_zones_with_mobility_station = gdf_mobilty_stations_with_npvm_zone.dissolve(by="ID", aggfunc={"N_Gem": "first", "Name": lambda x: list(x), "Stationsnummer": lambda x: list(x)}).reset_index()

In [17]:
print("Anzahl NPVM-Zonen mit Mobility-Standort: {}".format(len(gdf_npvm_zones_with_mobility_station)))

Anzahl NPVM-Zonen mit Mobility-Standort: 1311


In [18]:
gdf_npvm_zones_with_mobility_station.head(30)

,ID,geometry,N_Gem,Name,Stationsnummer
0,201004,POINT (8.44662 47.27617),Affoltern am Albis,[Affoltern a.A. Bahnhof],[4182]
1,201005,POINT (8.45109 47.27595),Affoltern am Albis,[Affoltern a.A. Zentrum / Areal Zürichstrasse],[2945]
2,201009,POINT (8.45641 47.28340),Affoltern am Albis,[Affoltern a.A. Bezirksspital],[1749]
3,301002,POINT (8.46839 47.32690),Bonstetten,[Bonstetten Bahnhof Bonstetten-Wettswil],[3226]
4,301003,POINT (8.47101 47.31595),Bonstetten,[Bonstetten Am Isenbach],[1532]
5,401001,POINT (8.53141 47.24452),Hausen am Albis,[Hausen am Albis Mülimatt],[1561]
6,501002,POINT (8.44536 47.29930),Hedingen,[Hedingen Bahnhof],[1563]
7,701002,POINT (8.46672 47.22007),Knonau,[Knonau Bahnhof],[2997]
8,901001,POINT (8.45705 47.24508),Mettmenstetten,[Mettmenstetten Bahnhof],[1582]
9,901003,POINT (8.46378 47.24320),Mettmenstetten,[Mettmenstetten Gemeindehaus],[4556]


### Read PT-skims

In [19]:
%time skim_jrta = ReadPTVMatrix(path_pt_jrta)

CPU times: total: 6.78 s
Wall time: 6.78 s


In [20]:
%time skim_ntr = ReadPTVMatrix(path_pt_ntr)

CPU times: total: 6.69 s
Wall time: 6.69 s


In [21]:
def get_skim(skim_matrix, from_npvm_zone_id, to_npvm_zone_id):
    return skim_matrix.sel(origins=from_npvm_zone_id).sel(destinations=to_npvm_zone_id).matrix.item()

In [22]:
def get_jrta(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(skim_jrta, from_npvm_zone_id, to_npvm_zone_id)

In [23]:
def get_ntr(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(skim_ntr, from_npvm_zone_id, to_npvm_zone_id)

In [24]:
get_jrta(378601001, 35101026)

286.0875843780647

In [25]:
get_ntr(378601001, 35101026)

3.000000000000001

## Execute query

### Define origin and destination and assign NPVM-Zone

In [26]:
orig_lon_lat = (7.423570, 46.936620)
dest_lon_lat = (7.695260, 46.828540)

In [27]:
orig_point = Point(orig_lon_lat[0], orig_lon_lat[1])
dest_point = Point(dest_lon_lat[0], dest_lon_lat[1])

In [28]:
gdf_orig = gpd.GeoDataFrame({'geometry': [orig_point]}, crs="EPSG:4326")
gdf_dest = gpd.GeoDataFrame({'geometry': [dest_point]}, crs="EPSG:4326")

In [29]:
gdf_orig_with_zone = gpd.sjoin(gdf_orig, gdf_npvm_zones)[["ID", "N_Gem", "geometry"]]
gdf_dest_zone = gpd.sjoin(gdf_orig, gdf_npvm_zones)[["ID", "N_Gem", "geometry"]]

In [30]:
gdf_orig_with_zone

,ID,N_Gem,geometry
0,35101026,Bern,POINT (7.42357 46.93662)


In [31]:
gdf_dest_zone

,ID,N_Gem,geometry
0,35101026,Bern,POINT (7.42357 46.93662)


In [32]:
orig_zone_id = gdf_orig_with_zone["ID"].item()

### Compute potential NPVM-zones with Mobility-station

In [33]:
gdf_circle_around_orig  = gpd.GeoDataFrame(geometry=gdf_orig_with_zone.to_crs(2026).buffer(5*1000).to_crs(4326), crs="EPSG:4326")
gdf_circle_around_dest = gpd.GeoDataFrame(geometry=gdf_dest_zone.to_crs(2026).buffer(50*1000).to_crs(4326), crs="EPSG:4326")

In [34]:
gdf_circle_around_orig

,geometry
0,"POLYGON ((7.40836 46.90462, 7.40386 46.90580, ..."


In [35]:
gdf_circle_around_dest

,geometry
0,"POLYGON ((7.27261 46.61729, 7.22799 46.62892, ..."


In [36]:
gdf_npvm_zones_with_mobility_station_in_orig_circle = gpd.sjoin(gdf_npvm_zones_with_mobility_station, gdf_circle_around_orig)
gdf_npvm_zones_with_mobility_station_in_dest_circle = gpd.sjoin(gdf_npvm_zones_with_mobility_station, gdf_circle_around_dest)

In [37]:
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Startpunkts: {}".format(len(gdf_npvm_zones_with_mobility_station_in_orig_circle)))
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Zielpunkts: {}".format(len(gdf_npvm_zones_with_mobility_station_in_dest_circle)))

Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Startpunkts: 63
Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Zielpunkts: 225


In [38]:
gdf_npvm_zones_with_mobility_station_in_orig_circle.head()

,ID,geometry,N_Gem,Name,Stationsnummer,index_right
347,35101004,"MULTIPOINT (7.38055 46.94513, 7.38364 46.94701)",Bern,"[Bern Migros Bethlehem, Bern Brünnen / Riedbac...","[2180, 4718]",0
348,35101006,POINT (7.38535 46.93170),Bern,[Bern Bümpliz / Wangenstrasse],[5248],0
349,35101007,POINT (7.38609 46.94300),Bern,[Bern Bümpliz / Brünnenstrasse],[4604],0
350,35101011,POINT (7.38905 46.94569),Bern,[Bern Bümpliz Nord Bahnhof],[2902],0
351,35101016,"MULTIPOINT (7.39396 46.94112, 7.39438 46.93700...",Bern,"[Bern Bümpliz Süd Bahnhof, Bern Bümpliz Zentru...","[2537, 3921, 4073]",0


In [39]:
gdf_npvm_zones_with_mobility_station_in_orig_circle.head()

,ID,geometry,N_Gem,Name,Stationsnummer,index_right
347,35101004,"MULTIPOINT (7.38055 46.94513, 7.38364 46.94701)",Bern,"[Bern Migros Bethlehem, Bern Brünnen / Riedbac...","[2180, 4718]",0
348,35101006,POINT (7.38535 46.93170),Bern,[Bern Bümpliz / Wangenstrasse],[5248],0
349,35101007,POINT (7.38609 46.94300),Bern,[Bern Bümpliz / Brünnenstrasse],[4604],0
350,35101011,POINT (7.38905 46.94569),Bern,[Bern Bümpliz Nord Bahnhof],[2902],0
351,35101016,"MULTIPOINT (7.39396 46.94112, 7.39438 46.93700...",Bern,"[Bern Bümpliz Süd Bahnhof, Bern Bümpliz Zentru...","[2537, 3921, 4073]",0


In [40]:
gdf_npvm_zones_with_potential_mobility_station = pd.concat([gdf_npvm_zones_with_mobility_station_in_orig_circle, gdf_npvm_zones_with_mobility_station_in_dest_circle]).groupby("ID").first().reset_index().drop(["index_right"], axis=1)

In [41]:
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Start- oder Zielpunkts: {}".format(len(gdf_npvm_zones_with_potential_mobility_station)))

Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Start- oder Zielpunkts: 225


In [42]:
gdf_npvm_zones_with_potential_mobility_station.head()

,ID,geometry,N_Gem,Name,Stationsnummer
0,30101004,POINT (7.27780 47.04357),Aarberg,[Aarberg Bahnhof],[2672]
1,30601005,POINT (7.30569 47.07726),Lyss,[Lyss Bahnhof],[1579]
2,30601007,POINT (7.30560 47.07900),Lyss,[Lyss Busswilstrasse],[4565]
3,30601012,POINT (7.31873 47.09890),Lyss,[Busswil Bahnhof],[4222]
4,31101002,POINT (7.38656 47.04189),Schüpfen,[Schüpfen Bahnhof],[1249]


In [43]:
gdf_npvm_zones_with_potential_mobility_station[gdf_npvm_zones_with_potential_mobility_station.ID == 30601007]

,ID,geometry,N_Gem,Name,Stationsnummer
2,30601007,POINT (7.30560 47.07900),Lyss,[Lyss Busswilstrasse],[4565]


### Get Mobility travel time and distance from ORMS

In [44]:
list_npvm_zones_with_potential_mobility_station = list(gdf_npvm_zones_with_potential_mobility_station.to_records())

In [45]:
list_npvm_zones_with_potential_mobility_station[:3]

[(0, 30101004, <POINT (7.278 47.044)>, 'Aarberg', list(['Aarberg Bahnhof']), list([2672])),
 (1, 30601005, <POINT (7.306 47.077)>, 'Lyss', list(['Lyss Bahnhof']), list([1579])),
 (2, 30601007, <POINT (7.306 47.079)>, 'Lyss', list(['Lyss Busswilstrasse']), list([4565]))]

In [46]:
coords_str = "{},{}".format(dest_point.x, dest_point.y)
for pot_mob_st in list_npvm_zones_with_potential_mobility_station:
    center = pot_mob_st[2].centroid
    coords_str += ";{},{}".format(center.x, center.y)
url = "https://router.project-osrm.org/table/v1/driving/{}?destinations=0&annotations=duration,distance".format(coords_str)

In [47]:
%time res = requests.get(url).json()

CPU times: total: 109 ms
Wall time: 403 ms


In [48]:
road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id = {x[1]: res["distances"][x[0] + 1][0] for x in list_npvm_zones_with_potential_mobility_station}
road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id = {x[1]: res["durations"][x[0] + 1][0] for x in list_npvm_zones_with_potential_mobility_station}

In [49]:
print("Anzahl Distanzen: {}".format(len(road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id)))
print("Anzahl Reisezeiten: {}".format(len(road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id)))

Anzahl Distanzen: 225
Anzahl Reisezeiten: 225


In [50]:
gdf_npvm_zones_with_potential_mobility_station.head()

,ID,geometry,N_Gem,Name,Stationsnummer
0,30101004,POINT (7.27780 47.04357),Aarberg,[Aarberg Bahnhof],[2672]
1,30601005,POINT (7.30569 47.07726),Lyss,[Lyss Bahnhof],[1579]
2,30601007,POINT (7.30560 47.07900),Lyss,[Lyss Busswilstrasse],[4565]
3,30601012,POINT (7.31873 47.09890),Lyss,[Busswil Bahnhof],[4222]
4,31101002,POINT (7.38656 47.04189),Schüpfen,[Schüpfen Bahnhof],[1249]


In [51]:
MIV_DISTANZ_BIS_ZIEL_KM = "MIV_Distanz_bis_Ziel_km"
MIV_ZEIT_BIS_ZIEL_MIN = "MIV_Zeit_bis_Ziel_min"

In [52]:
pd_distances = pd.DataFrame(list(road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id.items()), columns=["ID", "MIV_Distanz_bis_Ziel_km"])
pd_distances[MIV_DISTANZ_BIS_ZIEL_KM] = pd_distances[MIV_DISTANZ_BIS_ZIEL_KM] / 1000.0
pd_durations= pd.DataFrame(list(road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id.items()), columns=["ID", "MIV_Zeit_bis_Ziel_min"])
pd_durations[MIV_ZEIT_BIS_ZIEL_MIN] = pd_durations[MIV_ZEIT_BIS_ZIEL_MIN] / 60.0

gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station, pd_distances, on=["ID"])
gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_durations, on=["ID"])


In [53]:
gdf_npvm_zones_with_potential_mobility_station_with_data.head()

,ID,geometry,N_Gem,Name,Stationsnummer,MIV_Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min
0,30101004,POINT (7.27780 47.04357),Aarberg,[Aarberg Bahnhof],[2672],63.7240,52.573333
1,30601005,POINT (7.30569 47.07726),Lyss,[Lyss Bahnhof],[1579],65.1995,53.448333
2,30601007,POINT (7.30560 47.07900),Lyss,[Lyss Busswilstrasse],[4565],63.7379,53.451667
3,30601012,POINT (7.31873 47.09890),Lyss,[Busswil Bahnhof],[4222],68.2743,57.246667
4,31101002,POINT (7.38656 47.04189),Schüpfen,[Schüpfen Bahnhof],[1249],53.7185,46.515000


### Get pt skims from origin to potential Mobility stations

In [54]:
OEV_JRTA_VON_START_MIN = "OEV_JRTA_von_Start_min"
OEV_NTR_VON_START = "OEV_NTR_von_Start"

In [55]:
zone_ids_list = [x.item() for x in gdf_npvm_zones_with_potential_mobility_station[["ID"]].values]
jrta_list = [(x, get_jrta(orig_zone_id, x)) for x in zone_ids_list]
ntr_list = [(x, get_ntr(orig_zone_id, x)) for x in zone_ids_list]

In [56]:
pd_jrtas = pd.DataFrame(jrta_list, columns=["ID", OEV_JRTA_VON_START_MIN])
pd_ntrs = pd.DataFrame(ntr_list, columns=["ID", OEV_NTR_VON_START])

In [57]:
gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_jrtas, on=["ID"])
gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_ntrs, on=["ID"])

In [58]:
print(len(gdf_npvm_zones_with_potential_mobility_station_with_data))

225


In [59]:
KOSTEN_CHF = "Kosten_CHF"

In [60]:
CHF_PER_KM_MOBILITY = 0.75
MIN_PER_TRANSFER = 20.0
FILTER_FACTOR = 1.05

In [61]:
def calc_costs(vtt_chf_per_h=20):
    gdf_npvm_zones_with_potential_mobility_station_with_data[KOSTEN_CHF] = \
        CHF_PER_KM_MOBILITY * gdf_npvm_zones_with_potential_mobility_station_with_data[MIV_DISTANZ_BIS_ZIEL_KM] + \
        (gdf_npvm_zones_with_potential_mobility_station_with_data[MIV_ZEIT_BIS_ZIEL_MIN] + gdf_npvm_zones_with_potential_mobility_station_with_data[OEV_JRTA_VON_START_MIN] + 
        gdf_npvm_zones_with_potential_mobility_station_with_data[OEV_NTR_VON_START] * MIN_PER_TRANSFER) / 60.0 * vtt_chf_per_h
    return gdf_npvm_zones_with_potential_mobility_station_with_data.sort_values(by=KOSTEN_CHF, ascending=True)

In [62]:
gdf_npvm_zones_with_potential_mobility_station_with_data = calc_costs(25)

In [63]:
gdf_npvm_zones_with_potential_mobility_station_with_data.head()

,ID,geometry,N_Gem,Name,Stationsnummer,MIV_Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min,OEV_JRTA_von_Start_min,OEV_NTR_von_Start,Kosten_CHF
179,94201016,"MULTIPOINT (7.62172 46.76000, 7.62823 46.75613)",Thun,"[Thun Bahnhof, Thun Selveareal]","[1138, 4183]",12.2399,15.666667,45.913326,1.000087,43.172312
181,94201018,"MULTIPOINT (7.62289 46.75368, 7.62588 46.75568...",Thun,"[Thun Schlossmattstrasse, Thun Bahnhof / Parkh...","[4855, 5861, 5920]",12.1832,15.658333,46.963411,1.000086,43.563845
180,94201017,POINT (7.62580 46.74950),Thun,[Thun Hohmad],[4481],12.6738,16.148333,46.980079,1.000086,44.142907
14,35101026,POINT (7.42118 46.93880),Bern,[Bern Kirchbergerstrasse],[4662],39.2916,37.776667,0.000000,0.000000,45.208978
163,86101001,POINT (7.49383 46.89395),Belp,[Belp Lindenrain / Steinbach],[2712],30.3049,29.945000,29.379670,0.000000,47.447288


In [64]:
def get_best_mobility_stations_per_vtt(vtt_chf_per_h):
    df_tmp = calc_costs(vtt_chf_per_h=vtt_chf_per_h)
    min_cost = df_tmp[KOSTEN_CHF].min()
    df_tmp = df_tmp[df_tmp[KOSTEN_CHF] <= min_cost * FILTER_FACTOR]
    return pd.merge(gdf_mobilty_stations_with_npvm_zone, df_tmp["ID"], on="ID")

In [65]:
get_best_mobility_stations_per_vtt(25)

,Stationsnummer,Name,geometry,ID,N_Gem
0,1138,Thun Bahnhof,POINT (7.62823 46.75613),94201016,Thun
1,4183,Thun Selveareal,POINT (7.62172 46.76000),94201016,Thun
2,4481,Thun Hohmad,POINT (7.62580 46.74950),94201017,Thun
3,4662,Bern Kirchbergerstrasse,POINT (7.42118 46.93880),35101026,Bern
4,4855,Thun Schlossmattstrasse,POINT (7.62588 46.75568),94201018,Thun
5,5861,Thun Bahnhof / Parkhaus City Süd,POINT (7.62831 46.75467),94201018,Thun
6,5920,Thun Fliederweg,POINT (7.62289 46.75368),94201018,Thun


In [66]:
best_mobility_stations_per_vtt = {vtt: get_best_mobility_stations_per_vtt(vtt) for vtt in range(0, 101)}

In [67]:
best_mobility_stations_per_vtt[57]

,Stationsnummer,Name,geometry,ID,N_Gem
0,4662,Bern Kirchbergerstrasse,POINT (7.42118 46.93880),35101026,Bern


In [68]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Steffisburg"]

,ID,geometry,N_Gem,Name,Stationsnummer,MIV_Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min,OEV_JRTA_von_Start_min,OEV_NTR_von_Start,Kosten_CHF
171,93901006,POINT (7.63106 46.77080),Steffisburg,[Steffisburg Ziegelei],[1123],9.9043,11.820000,68.014571,2.0,207.152509
173,93901010,POINT (7.63644 46.77829),Steffisburg,[Steffisburg Höchhusweg],[5006],9.0672,10.671667,71.815261,2.0,210.945279
172,93901007,POINT (7.63479 46.78196),Steffisburg,[Steffisburg Sunnegg],[1884],8.6377,10.198333,78.483295,2.0,220.947656


In [69]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Thun"]

,ID,geometry,N_Gem,Name,Stationsnummer,MIV_Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min,OEV_JRTA_von_Start_min,OEV_NTR_von_Start,Kosten_CHF
179,94201016,"MULTIPOINT (7.62172 46.76000, 7.62823 46.75613)",Thun,"[Thun Bahnhof, Thun Selveareal]","[1138, 4183]",12.2399,15.666667,45.913326,1.000087,145.149472
181,94201018,"MULTIPOINT (7.62289 46.75368, 7.62588 46.75568...",Thun,"[Thun Schlossmattstrasse, Thun Bahnhof / Parkh...","[4855, 5861, 5920]",12.1832,15.658333,46.963411,1.000086,146.843181
180,94201017,POINT (7.62580 46.74950),Thun,[Thun Hohmad],[4481],12.6738,16.148333,46.980079,1.000086,148.055578
176,94201010,POINT (7.61361 46.75667),Thun,[Thun Energie Thun AG],[3782],12.2229,16.946667,55.022855,2.009723,196.107153
178,94201014,POINT (7.62483 46.74064),Thun,[Thun Dürrenast],[2142],13.8524,17.673333,64.285229,2.003457,213.768817
175,94201009,POINT (7.61651 46.74410),Thun,[Thun Talacker],[4236],13.8218,17.933333,69.915924,2.000000,223.448445
177,94201013,POINT (7.62106 46.73390),Thun,[Thun Gwatt / Schorenstrasse],[4626],14.7049,18.981667,69.569883,2.000000,225.281257
174,94201008,POINT (7.61516 46.73598),Thun,[Thun Bostudenstrasse],[1849],15.1392,20.143333,72.740559,2.000000,232.827553


In [70]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Bern"].head(50)

,ID,geometry,N_Gem,Name,Stationsnummer,MIV_Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min,OEV_JRTA_von_Start_min,OEV_NTR_von_Start,Kosten_CHF
14,35101026,POINT (7.42118 46.93880),Bern,[Bern Kirchbergerstrasse],[4662],39.2916,37.776667,0.000000,0.000000,92.429811
18,35101031,"MULTIPOINT (7.42792 46.94227, 7.42864 46.94072)",Bern,"[Bern Tscharnerstrasse, Bern Lentulusrain]","[4593, 4921]",38.5713,36.665000,9.668588,0.002109,106.221418
28,35101048,POINT (7.43641 46.94440),Bern,[Bern Kapellenstrasse],[4652],38.4050,36.001667,11.462131,0.002104,107.980216
32,35101058,POINT (7.44764 46.94660),Bern,[Bern Parking Casino],[4534],37.6392,35.383333,14.073889,0.001852,110.719852
34,35101060,POINT (7.44681 46.94950),Bern,[Bern Nägeligasse],[3795],37.9076,35.468333,14.073889,0.001852,111.062818
16,35101028,"MULTIPOINT (7.41906 46.94070, 7.42777 46.94004)",Bern,"[Bern Lentulusstrasse, Bern Fischermätteli / W...","[4274, 4454]",39.0687,37.393333,10.950000,0.000000,109.873747
31,35101055,"MULTIPOINT (7.44335 46.94518, 7.44356 46.94460...",Bern,"[Bern Marzili Talstation, Bern Marzili / Dampf...","[1527, 2737, 4601]",37.9357,36.371667,14.073889,0.001852,112.589449
26,35101045,POINT (7.43417 46.94623),Bern,[Bern Parking City West],[2017],38.7490,36.380000,13.390202,0.009489,112.328372
29,35101049,"MULTIPOINT (7.43679 46.95050, 7.43770 46.94950...",Bern,"[Bern Bahnhof, Bern Sidlerstrasse, Bern Gesell...","[2687, 4167, 4198]",39.0757,37.840000,14.073889,0.001852,115.891671
39,35101070,"MULTIPOINT (7.44987 46.94914, 7.45178 46.94900)",Bern,"[Bern Altstadt / Brunngasshalde, Bern Rathausp...","[3153, 4530]",37.4551,35.353333,19.688874,0.088459,122.776968


### Visualize situation on map

In [71]:
bounds = gdf_npvm_zones_with_potential_mobility_station_with_data.total_bounds
bounds = [[bounds[1], bounds[0]], [bounds[3], bounds[2]]]
bounds

[[46.66, 6.94714], [47.233933, 7.78534]]

In [72]:
best_mobility_stations_per_vtt[20]

,Stationsnummer,Name,geometry,ID,N_Gem
0,1138,Thun Bahnhof,POINT (7.62823 46.75613),94201016,Thun
1,4183,Thun Selveareal,POINT (7.62172 46.76000),94201016,Thun
2,4481,Thun Hohmad,POINT (7.62580 46.74950),94201017,Thun
3,4855,Thun Schlossmattstrasse,POINT (7.62588 46.75568),94201018,Thun
4,5861,Thun Bahnhof / Parkhaus City Süd,POINT (7.62831 46.75467),94201018,Thun
5,5920,Thun Fliederweg,POINT (7.62289 46.75368),94201018,Thun


In [106]:
def show_best_mobility_stations_per_vtt(vtt):
    geo_data = GeoData(geo_dataframe=best_mobility_stations_per_vtt[vtt],
        style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
        hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
        point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
        name = 'Mobility-Stationen')
    for l in m.layers:
        if type(l) == GeoData:
            m.remove_layer(l)
    m.add_layer(geo_data)

In [107]:
def on_slider_changed(event):
    show_best_mobility_stations_per_vtt(event["new"])

In [108]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl, FullScreenControl, Marker, WidgetControl
from ipywidgets import IntSlider
import geopandas
import json

vtt_slider = IntSlider(description='Zeitkosten', min=0, max=100, value=20)
vtt_slider.observe(on_slider_changed, names='value')

widget_control = WidgetControl(widget=vtt_slider, position='topright')

m = Map(center=(52.3,8.0), zoom = 3, basemap= basemaps.Esri.WorldTopoMap, scroll_wheel_zoom=True)
m.add_control(widget_control)

m.fit_bounds(bounds)
show_best_mobility_stations_per_vtt(vtt_slider.value)
m.add_control(FullScreenControl())

m.layout.width = '100%'
m.layout.height = '500px'

orig_marker = Marker(location=(orig_lon_lat[1], orig_lon_lat[0]) , draggable=False)
dest_marker = Marker(location=(dest_lon_lat[1], dest_lon_lat[0]) , draggable=False)

m.add_layer(orig_marker)
m.add_layer(dest_marker)

m

Map(center=[52.3, 8.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…